In [11]:

from openai import OpenAI
import os, json
import httpx

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"),http_client=httpx.Client(verify=False))


In [ ]:
GOOD_BULLETS_SCHEMA = {
  "name": "good_bullets",
  "schema": {
    "type": "object",
    "properties": {
      "bullets": {
        "type": "array",
        "items": {"type": "string"}
      }
    },
    "required": ["bullets"],
    "additionalProperties": False
  }
}

payload = {
  "chem_type": "silver nanoparticle (AgNP) synthesis in a flow system",
  "objective": "loss (minimize)",
  "parameters": [
    {"name":'moleq2', "low":1.0, "high":43.57, "unit":'%', "role":'acetone equivalents', "type":'continuous'},
    {"name":'moleq3', "low":0.02, "high":0.2, "unit":'%', "role":'Base equivalents', "type":'continuous'},
    {"name":'temp', "low":30, "high":70, "unit":'celsius', "role":'Reaction temperature', "type":'continuous'},
    {"name":'time', "low":5, "high":15, "unit":'mins', "role":'residence time (Flow chemistry)', "type":'continuous'}
  ],
  "k_bullets": 10
}


resp = client.responses.create(
  model="gpt-4o",
  instructions=(
    "You are a chemistry literature agent. Use web_search to find evidence. "
    "Return ONLY JSON with key 'bullets', where 'bullets' is a list of context strings. "
    "Each string should be one short, evidence-grounded optimization hint."
    "no links, no citations, no quotes, just the bullet points."
  ),
  input=[{
    "role": "user",
    "content": [{"type":"input_text","text": json.dumps(payload)}]
  }],
  tools=[{"type":"web_search"}],
  text={
    "format": {
      "type": "json_schema",
      "name": GOOD_BULLETS_SCHEMA["name"],
      "schema": GOOD_BULLETS_SCHEMA["schema"],
      "strict": False  # you can flip to True if you want hard enforcement
    }
  }
)

out = resp.output_text
parsed = json.loads(out)
GOOD_BULLETS = parsed.get("bullets", [])
print(GOOD_BULLETS)

['Use a slight excess of benzaldehyde relative to acetone (e.g., ~2:1 ratio) to drive the crossed aldol condensation toward benzylideneacetone and minimize self‑condensation of acetone', 'Operate in a protic solvent like ethanol or water/ethanol mixture to enhance enolate formation yet allow precipitation of product for easy isolation', 'Maintain moderate base loading—just enough NaOH to generate acetone enolate but avoid excessive base that may promote retro‑aldol or Cannizzaro side reactions', 'Optimize reaction temperature within a moderate range (e.g., 30‑70\u202f°C) to balance enolate formation rate with minimizing side reactions and unwanted polymerization', 'Use short residence time (5–15\u202fmin) in flow to enhance productivity while minimizing retro‑aldol degradation of the product', 'Efficient mixing of acetone enolate with benzaldehyde—such as pre‑mixing base and acetone then gradual addition of benzaldehyde—helps favor the desired crossed product', 'Encourage precipitation

In [23]:
GOOD_BULLETS

['Use a slight excess of benzaldehyde relative to acetone (e.g., ~2:1 ratio) to drive the crossed aldol condensation toward benzylideneacetone and minimize self‑condensation of acetone',
 'Operate in a protic solvent like ethanol or water/ethanol mixture to enhance enolate formation yet allow precipitation of product for easy isolation',
 'Maintain moderate base loading—just enough NaOH to generate acetone enolate but avoid excessive base that may promote retro‑aldol or Cannizzaro side reactions',
 'Optimize reaction temperature within a moderate range (e.g., 30‑70\u202f°C) to balance enolate formation rate with minimizing side reactions and unwanted polymerization',
 'Use short residence time (5–15\u202fmin) in flow to enhance productivity while minimizing retro‑aldol degradation of the product',
 'Efficient mixing of acetone enolate with benzaldehyde—such as pre‑mixing base and acetone then gradual addition of benzaldehyde—helps favor the desired crossed product',
 'Encourage precipi